In [5]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import datetime as dt

# Import data
hsx = pd.read_csv('CafeF.HSX.Upto06.09.2024.csv', parse_dates=['<DTYYYYMMDD>'])
hnx = pd.read_csv('CafeF.HNX.Upto06.09.2024.csv', parse_dates=['<DTYYYYMMDD>'])
upcom = pd.read_csv('CafeF.UpCom.Upto06.09.2024.csv', parse_dates=['<DTYYYYMMDD>'])

# 1Y data
date_filter = hsx['<DTYYYYMMDD>'].unique()[:252]
hsx_252 = hsx[hsx['<DTYYYYMMDD>'].isin(date_filter)]
hnx_252 = hnx[hnx['<DTYYYYMMDD>'].isin(date_filter)]
upcom_252 = upcom[upcom['<DTYYYYMMDD>'].isin(date_filter)]

# Data processing
df_raw = pd.concat([hsx_252, hnx_252, upcom_252])
df_raw.rename(columns = {'<DTYYYYMMDD>': 'Date', '<Ticker>': 'Ticker', '<Close>': 'Close','<Volume>':'Volume', '<Open>': 'Open', '<High>' : 'High', '<Low>' : 'Low'}, inplace=True)
df_raw.sort_values(by = ['Ticker','Date'], ascending=True, inplace=True)
df_raw['Ticker length'] = df_raw['Ticker'].str.len()
df = df_raw[df_raw['Ticker length'] < 4].drop(labels = 'Ticker length', axis = 1)
df=df.reset_index(drop=True)

In [35]:
##### MFI đi vào vùng quá mua/bán
# MFI đi vào vùng quá mua
def mfi_move_into_overbuy():
    df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['MFI Overbuy'] = df['mfi'] > 80
    mfi_yesterday = df[(df['Date'] == yesterday) & (df['MFI Overbuy'] == False)][['Ticker']]
    mfi_today = df[(df['Date'] == today) & (df['MFI Overbuy'] == True)][['Ticker']]
    mfi_ticker_overbuy = mfi_today.merge(mfi_yesterday, on = 'Ticker')
    return mfi_ticker_overbuy['Ticker'].unique()

#MFI đi vào vùng quá bán
def mfi_move_into_oversell():
    df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['MFI Oversell'] = df['mfi'] < 20
    mfi_yesterday = df[(df['Date'] == yesterday) & (df['MFI Oversell'] == False)][['Ticker']]
    mfi_today = df[(df['Date'] == today) & (df['MFI Oversell'] == True)][['Ticker']]
    mfi_ticker_overbuy = mfi_today.merge(mfi_yesterday, on = 'Ticker')
    return mfi_ticker_overbuy['Ticker'].unique()


#####MFI thoát khỏi vùng quá mua/bán
# MFI thoát khỏi vùng quá mua
def mfi_move_out_overbuy():
    df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['MFI Overbuy'] = df['mfi'] > 80
    mfi_yesterday = df[(df['Date'] == yesterday) & (df['MFI Overbuy'] == True)][['Ticker']]
    mfi_today = df[(df['Date'] == today) & (df['MFI Overbuy'] == False)][['Ticker']]
    mfi_ticker_overbuy = mfi_today.merge(mfi_yesterday, on = 'Ticker')
    return mfi_ticker_overbuy['Ticker'].unique()

# MFI thoát khỏi vùng quá bán
def mfi_move_out_oversell():
    df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['MFI Overbuy'] = df['mfi'] < 20
    mfi_yesterday = df[(df['Date'] == yesterday) & (df['MFI Overbuy'] == True)][['Ticker']]
    mfi_today = df[(df['Date'] == today) & (df['MFI Overbuy'] == False)][['Ticker']]
    mfi_ticker_overbuy = mfi_today.merge(mfi_yesterday, on = 'Ticker')
    return mfi_ticker_overbuy['Ticker'].unique()


#####MFI đang ở trong vùng quá mua/bán
# MFI đang ở trong vùng quá mua 2 ngày liên tục
def mfi_in_overbuy():
    df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['MFI Overbuy'] = df['mfi'] > 80
    mfi_yesterday = df[(df['Date'] == yesterday) & (df['MFI Overbuy'] == True)][['Ticker']]
    mfi_today = df[(df['Date'] == today) & (df['MFI Overbuy'] == True)][['Ticker']]
    mfi_ticker_overbuy = mfi_today.merge(mfi_yesterday, on = 'Ticker')
    return mfi_ticker_overbuy['Ticker'].unique()

# MFI đang ở trong vùng quá bán 2 ngày liên tục
def mfi_in_oversell():
    df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['MFI Overbuy'] = df['mfi'] < 20
    mfi_yesterday = df[(df['Date'] == yesterday) & (df['MFI Overbuy'] == True)][['Ticker']]
    mfi_today = df[(df['Date'] == today) & (df['MFI Overbuy'] == True)][['Ticker']]
    mfi_ticker_overbuy = mfi_today.merge(mfi_yesterday, on = 'Ticker')
    return mfi_ticker_overbuy['Ticker'].unique()

In [49]:
mfi_in_oversell()

C:\Users\MSI-PC\AppData\Local\Temp\ipykernel_14540\288722699.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[6.23005067e+03 2.98917667e+03 8.90414000e+01 ... 2.46410200e+05
 5.28025100e+05 8.22551600e+05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)
C:\Users\MSI-PC\AppData\Local\Temp\ipykernel_14540\288722699.py:63: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[14824.5065     14779.83333333 10374.56       ... 61985.
 90936.3        13662.        ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['mfi'] = ta.mfi(df['High'], df['Low'], df['Close'], df['Volume'], length=14)


array(['APC', 'APH', 'APP', 'ASG', 'BCE', 'BIC', 'BMJ', 'BT1', 'CDC',
       'CI5', 'CQT', 'DAS', 'DC2', 'DTG', 'DVP', 'E29', 'EPC', 'FIC',
       'GEE', 'HAV', 'HDA', 'HSM', 'ISH', 'LHC', 'LPT', 'MGR', 'MIC',
       'ND2', 'NDX', 'PCT', 'PDN', 'PSE', 'PTD', 'PV2', 'SDD', 'SDK',
       'SJM', 'TMP', 'TMS', 'TRA', 'TTD', 'UNI', 'VBC', 'VNP', 'VSH',
       'VTA', 'XMC'], dtype=object)